# Seleccion de K y Clustering con K-Means

## Objetivos del Cuaderno

1. **Selección del Número de Clusters (K)**: Probar diferentes metodos para obtener valores de K y determinar el número óptimo de clusters.
    - **Método del Codo**: Graficar la inercia en función de K y buscar el "codo" en la gráfica.
    - **Silhouette Score**: Calcular el coeficiente de silueta para diferentes valores de K y graficar los resultados.
    - **Davies-Bouldin Index**: Calcular el índice de Davies-Bouldin para diferentes valores de K y graficar los resultados.
    - **K-ISAC-TLP**: Utilizar el método K-ISAC-TLP para determinar el número óptimo de clusters.
        - **ISAC Curves**: Graficar las curvas ISAC para diferentes valores de K y observar la tendencia.
        - **MAE**: Calcular el error absoluto medio (MAE) para diferentes valores de K y graficar los resultados.
        - **Irrelevant clusters**: Identificar y eliminar clusters irrelevantes que no aporten información significativa.
2. **Clustering con Clustering Spectral**: Aplicar el algoritmo de Clustering Spectral para agrupar las viviendas en función de sus patrones de consumo energético.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score, mean_absolute_error, davies_bouldin_score
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("dataset/features_StandardScaler.csv", index_col='cups')

## Selección del Número de Clusters (K)

La elección del número óptimo de clusters (K) es un paso crucial en el algoritmo K-Means. Un valor de K demasiado bajo puede agrupar datos inherentemente distintos, mientras que un valor demasiado alto puede dividir clusters significativos o crear clusters con muy pocos miembros.

En este análisis, evaluamos varios valores de K dentro de un rango predefinido (2 a 30) utilizando las siguientes métricas:

* **Índice de Silueta:** Evalúa qué tan bien cada punto se ajusta a su propio cluster en comparación con otros clusters. Varía de -1 a 1, donde:
    * 1 indica que el punto está bien agrupado.
    * 0 indica que el punto está cerca del límite de decisión entre dos clusters.
    * -1 indica que el punto podría estar mejor en el cluster vecino.

* **Índice de Davies-Bouldin (DBI):** Mide la similitud promedio entre cada cluster y su cluster "más similar". Un DBI más bajo indica una mejor separación entre clusters.

**Metodología:**

1.  **Carga de Datos:** Se carga el dataset preprocesado y escalado (StandardScaler).
2.  **Iteración sobre Valores de K:** Para cada valor de K en el rango especificado:
    * Se instancia y entrena un modelo KMeans con el valor de K actual.
    * Se obtienen las etiquetas de los clusters
    * Se calculan las métricas de indice de Silueta y DBI.

**Interpretación de las Métricas:**

* Se busca un valor de K que minimice el DBI, y maximice el Índice de Silueta.
* Un número elevado de clusters irrelevantes sugiere que K es demasiado grande.
* El Índice de Silueta cercano a 1 indica que los clusters están bien separados.
* El Índice de Davies-Bouldin bajo indica que los clusters son distintos y compactos.

### ¿Cómo funciona el **clustering espectral**?

1. **Matriz de similitud (afinidad):**  
   Se construye una matriz `S` donde cada elemento $S_{ij}$ representa la similitud entre la vivienda $i$ y la $j$.  
   Esto se puede calcular, por ejemplo, con:
   - **RBF kernel (gaussiano):**
     $$
     S_{ij} = \exp\left(-\frac{||x_i - x_j||^2}{2\sigma^2}\right)
     $$

2. **Matriz Laplaciana:**  
   A partir de la matriz de similitud $S$, se construye la matriz Laplaciana:
   $$
   L = D - S
   $$
   o bien la versión normalizada:
   $$
   L_{\text{sym}} = D^{-1/2} L D^{-1/2}
   $$
   donde $D$ es la matriz diagonal de grados, con $D_{ii} = \sum_j S_{ij}$.

3. **Descomposición espectral:**  
   Se calculan los **autovectores** asociados a los menores **autovalores** de la matriz Laplaciana.  
   Estos vectores forman una nueva representación de menor dimensión del conjunto de datos.

4. **Clustering sobre el espacio espectral:**  
   Finalmente, se aplica **K-means** sobre las filas de esta nueva matriz espectral.  
   Cada fila representa una vivienda transformada en este nuevo espacio de características, donde las relaciones de similitud son más evidentes.

### ¿Por qué usar **clustering espectral** en lugar de solo K-means?

Porque el clustering espectral:

- **No asume formas esféricas** ni separación lineal entre clústeres.
- Puede capturar **relaciones no lineales** complejas en los datos.
- Es ideal cuando:
  - Hay **clases que se solapan** parcialmente.
  - Existen **patrones temporales no triviales** (como en series de consumo eléctrico).
  - Los clústeres tienen **formas irregulares o extendidas**.

### Affinity Propagation y Nearest Neighbors

In [3]:
X = df.values

param_grid = [
    {
        'affinity': 'rbf',
        'gamma_values': [0.1, 0.5, 1.0, 2.0, 5.0],
        'n_clusters_range': range(2, 15)
    },
    {
        'affinity': 'nearest_neighbors',
        'n_neighbors_values': [5, 10, 15, 20, 25],
        'n_clusters_range': range(2, 15)
    }
]

results = []

for config in param_grid:
    if config['affinity'] == 'rbf':
        for gamma in config['gamma_values']:
            for k in config['n_clusters_range']:
                try:
                    model = SpectralClustering(
                        n_clusters=k,
                        affinity=config['affinity'],
                        gamma=gamma,
                        random_state=42
                    )
                    labels = model.fit_predict(X)
                    unique_clusters = len(np.unique(labels))
                    
                    if unique_clusters == k:
                        silhouette = silhouette_score(X, labels)
                        davies_bouldin = davies_bouldin_score(X, labels)
                        
                        results.append({
                            'method': 'RBF',
                            'n_clusters': k,
                            'param_name': 'gamma',
                            'param_value': gamma,
                            'clusters_obtenidos': unique_clusters,
                            'silhouette': silhouette,
                            'davies_bouldin': davies_bouldin
                        })
                except Exception as e:
                    print(f"Error con {config['affinity']}, k={k}, gamma={gamma}: {e}")
    
    elif config['affinity'] == 'nearest_neighbors':
        for n_neighbors in config['n_neighbors_values']:
            for k in config['n_clusters_range']:
                try:
                    model = SpectralClustering(
                        n_clusters=k,
                        affinity=config['affinity'],
                        n_neighbors=n_neighbors,
                        random_state=42
                    )
                    labels = model.fit_predict(X)
                    unique_clusters = len(np.unique(labels))
                    
                    if unique_clusters == k:
                        silhouette = silhouette_score(X, labels)
                        davies_bouldin = davies_bouldin_score(X, labels)
                        
                        results.append({
                            'method': 'NearestNeighbors',
                            'n_clusters': k,
                            'param_name': 'n_neighbors',
                            'param_value': n_neighbors,
                            'clusters_obtenidos': unique_clusters,
                            'silhouette': silhouette,
                            'davies_bouldin': davies_bouldin
                        })
                except Exception as e:
                    print(f"Error con {config['affinity']}, k={k}, n_neighbors={n_neighbors}: {e}")




### Analisis de Resultados

El siguiente paso consiste en analizar los resultados (Índice de Silueta, DBI) para determinar el valor óptimo de K. Esto puede implicar la visualización de las métricas en función de K y la selección del valor que proporcione el mejor equilibrio entre las diferentes métricas.  Posteriormente, se podría aplicar un algoritmo como ISAC para refinar aún más la selección de K.

In [4]:
results_df = pd.DataFrame(results)

print(results_df.sort_values(by='silhouette', ascending=False).head())

   method  n_clusters param_name  param_value  clusters_obtenidos  silhouette  \
13    RBF           2      gamma          0.5                   2    0.592835   
26    RBF           2      gamma          1.0                   2    0.592835   
28    RBF           2      gamma          2.0                   2    0.592835   
27    RBF           3      gamma          1.0                   3    0.581774   
29    RBF           2      gamma          5.0                   2    0.513702   

    davies_bouldin  
13        0.294085  
26        0.294085  
28        0.294085  
27        0.261824  
29        0.347824  


In [5]:
print(results_df.sort_values(by='davies_bouldin', ascending=True).head())

   method  n_clusters param_name  param_value  clusters_obtenidos  silhouette  \
27    RBF           3      gamma          1.0                   3    0.581774   
13    RBF           2      gamma          0.5                   2    0.592835   
26    RBF           2      gamma          1.0                   2    0.592835   
28    RBF           2      gamma          2.0                   2    0.592835   
29    RBF           2      gamma          5.0                   2    0.513702   

    davies_bouldin  
27        0.261824  
13        0.294085  
26        0.294085  
28        0.294085  
29        0.347824  


### Mejor Configuración de Hiperparámetros

| N° Clusters | Afinidad | Gamma | Silhouette Score  | Davies-Bouldin Score  |
|-------------|----------|-------|-------------------|-----------------------|
| 12          | rbf      | 2.0   | 0.593             | 0.22940859            |
| 7           | rbf      |  2.0  | 0.592835          | 0.294085              |

### Interpretación de Métricas

#### Silhouette Score (0.593)
- **Rango óptimo**: [0.5, 1.0]
- **Interpretación**: 
  - Estructura de clusters significativa
  - Los clusters están razonablemente separados
  - Los puntos están bien asignados a sus clusters

#### Davies-Bouldin Score (0.294)
- **Rango óptimo**: [0, 1] (menor es mejor)
- **Interpretación**:
  - Excelente separación entre clusters
  - Los centroides están bien distanciados
  - Resultado sobresaliente (valores < 0.5 son considerados buenos)

## Aplicación de Spectral Clustering con el Número Óptimo de Clusters

In [6]:
best_k = 3
gamma = 1.0

sc_final = SpectralClustering(
                n_clusters=best_k,
                affinity='rbf',
                gamma=gamma,
                random_state=42
            )
sc_final.fit(X)

labels = sc_final.fit_predict(X)

print(len(np.unique(labels)))

3


## Guardar el Modelo Final

In [7]:
with open("pkls/spectral_clustering_model.pkl", "wb") as f:
    pickle.dump(sc_final, f)